In [1]:
#from google.colab import drive
#drive.mount('/content/Drive')

In [2]:
import pandas as pd
#df = pd.read_csv("/content/Drive/MyDrive/output1.txt", sep="|", header=None, names=["text", "rephrase"])
# print(df.head())

df = pd.read_csv("./output.txt", sep="|", header=None, names=["text", "rephrase"])
print(df.head())

                                                text  \
0  I need to get away from this city for a bit. I...   
1  Hate when people try to guilt trip you into so...   
2  Why does it have to be so hard to network if y...   
3  Children with autism use stereotypic behavior ...   
4  I have less than 2 hours to get home and make ...   

                                            rephrase  
0  I'm thankful for everything this city gave me!...  
1  I don’t like it when people try to guilt -trip...  
2  I'm glad networking with gaming comes very eas...  
3  Children with autism tend to demonstrate stere...  
4  I have less than 2 hours to get home and make ...  


In [3]:
!pip3 install transformers
!pip3 install Datasets
!pip3 install evaluate
!pip3 install rouge_score
!pip3 install sacrebleu
!pip3 install transformers -U
!pip3 install accelerate -U
#!pip install torch -U

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to us

In [4]:
import argparse
import pandas as pd
import numpy as np
import random
#from sentence_transformers import SentenceTransformer, util
import os
from transformers import Trainer, pipeline, set_seed, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import nltk
nltk.download('punkt')
import csv
from datasets import load_dataset, load_metric, DatasetDict, Dataset


/Users/snehadarshini/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/snehadarshini/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
from transformers import BartForConditionalGeneration, BartTokenizer

model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)


In [6]:
# Example data preparation
input_texts = ["The quick brown fox.", "A sentence to be rephrased."]
positive_rephrases = ["A fast brown fox.", "A differently phrased sentence."]

input_encodings = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)
positive_encodings = tokenizer(positive_rephrases, return_tensors="pt", padding=True, truncation=True)


In [7]:
from sklearn.model_selection import train_test_split

# Assuming your dataframe is named 'df'
# Specify the test size as 0.05 to get a 95%/5% train/test split
train_df, test_df = train_test_split(df, test_size=0.10, random_state=42)

In [8]:
train_data = train_df.iloc[0:4000]
val_data = train_df.iloc[4000:6011]

In [9]:
# convert to dataset
raw_data = DatasetDict({
    "train": Dataset.from_pandas(train_data),
    "eval": Dataset.from_pandas(val_data),
    "test": Dataset.from_pandas(test_df)
})

In [10]:
metric = load_metric("rouge")
metric2 = load_metric("sacrebleu")

/var/folders/t3/0fprr8zn0vs0pk4bn0xd0r780000gn/T/ipykernel_43668/1803931011.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [11]:
def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Rouge expects a newline after each sentence
        decoded_preds_joined = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
        decoded_labels_joined = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

        result = metric.compute(predictions=decoded_preds_joined, references=decoded_labels_joined, use_stemmer=True)
        # Extract a few results
        #result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
        result = {key: value.mid.fmeasure for key, value in result.items()}

        # Add mean generated length
        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
        result["gen_len"] = np.mean(prediction_lens)

        # Add bleu scores
        decoded_labels_expanded = [[x] for x in decoded_labels]
        result2 = metric2.compute(predictions=decoded_preds, references=decoded_labels_expanded)
        result['sacrebleu'] = round(result2["score"], 1)

        return {k: round(v, 4) for k, v in result.items()}

In [12]:
def preprocess_function(row):
    inputs = row["text"]
    model_inputs = tokenizer(inputs, max_length=64, truncation=True, padding='max_length')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(row["rephrase"], max_length=64, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
tokenize_data = raw_data.map(preprocess_function, batched = True, remove_columns=['text', 'rephrase'])

Map:   0%|                                      | 0/4000 [00:00<?, ? examples/s]/Users/snehadarshini/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|███████████████████████████| 668/668 [00:00<00:00, 5874.03 examples/s]


In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [15]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
batch_size = 1
# Example fine-tuning configuration
training_args = Seq2SeqTrainingArguments(
    output_dir="output/bart-cnn-rephrase",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size= batch_size,
    save_total_limit=2,
    num_train_epochs=2,
    predict_with_generate=True,
    overwrite_output_dir=True,
    fp16=False,
    logging_dir="./logs",
)




In [16]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['eval'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics

)


In [17]:
trainer.train()


Step,Training Loss
500,1.096900
1000,1.123200
1500,1.077300
2000,0.961600
2500,0.896600
3000,0.902300
3500,0.875800
4000,0.860600
4500,0.599700
5000,0.611600


TrainOutput(global_step=8000, training_loss=0.7790583000183106, metrics={'train_runtime': 5439.8386, 'train_samples_per_second': 1.471, 'train_steps_per_second': 1.471, 'total_flos': 1083552301056000.0, 'train_loss': 0.7790583000183106, 'epoch': 2.0})

In [18]:
trainer.save_model("bart-cnn/bart-cnn-rephrase")

In [19]:
trainer.evaluate()

{'eval_loss': 0.8330714106559753,
 'eval_rouge1': 0.2708,
 'eval_rouge2': 0.1052,
 'eval_rougeL': 0.2072,
 'eval_rougeLsum': 0.2309,
 'eval_gen_len': 60.0467,
 'eval_sacrebleu': 5.4,
 'eval_runtime': 3914.671,
 'eval_samples_per_second': 0.514,
 'eval_steps_per_second': 0.514,
 'epoch': 2.0}

In [20]:
from transformers import BartForConditionalGeneration, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained("bart-cnn/bart-cnn-rephrase")
tokenizer = AutoTokenizer.from_pretrained("bart-cnn/bart-cnn-rephrase")
positivereframer = pipeline('summarization', model=model, tokenizer=tokenizer)
texts = raw_data['test']['text']

In [21]:
positivereframer("Southend Hospital was like a cross between Argos and Southendairport today Everybody looked SO and sad! NOT a place toGET well")

Your max_length is set to 142, but your input_length is only 32. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)


[{'summary_text': "Southend Hospital was like a cross between Argos and Southendairport today. Everybody looked sad, but I'm thankful that I have a hospital to go to, and I'm in a good place to get well and get on with my life."}]

In [27]:
reframed_phrases = [positivereframer(text)[0]['summary_text'] for text in texts]

Your max_length is set to 142, but your input_length is only 34. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)
Your max_length is set to 142, but your input_length is only 18. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)
Your max_length is set to 142, but your input_length is only 24. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)
Your max_length is set to 142, but your input_length is only 35. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)
Your 

In [34]:
len(reframed_phrases)

668

In [38]:
data = {"original": raw_data['test']['rephrase'], "rephrased": reframed_phrases}

In [45]:
df = pd.DataFrame(data)

In [46]:
df.to_csv('bart-cnn/bart-output/test-bart-cnn-large-rephrased.csv', index=False)

In [35]:
with open(os.path.join("bart-cnn/output/bart-output.txt"), 'w') as f:
        for item in reframed_phrases:
            f.write("%s\n" % item)

In [49]:
texts = raw_data['test']['text']

with open(os.path.join("bart-cnn/bart-output/bart-input.txt"), 'w') as f:
        for item in texts:
            f.write("%s\n" % item)

In [37]:
texts = raw_data['test']['rephrase']

with open(os.path.join("bart-cnn/bart-output/bart-original-rephrased.txt"), 'w') as f:
        for item in texts[:20]:
            f.write("%s\n" % item)